In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Bidirectional, LSTM, Dense, Concatenate, Flatten
from tensorflow.keras.models import Model

# --- !! This MUST match the number from Day 2 !! ---
SIGNAL_WINDOW_SIZE = 100
# --------------------------------------------------

# Load the data you saved
print("Loading data...")
X_train = np.load('X_train_ultra.npy')
y_train = np.load('y_train_ultra.npy')
X_test = np.load('X_test_ultra.npy')
y_test = np.load('y_test_ultra.npy')

# --- CRITICAL STEP: Reshaping ---
# Keras 1D CNNs expect a 3D shape: (samples, steps, channels)
# Your data is (samples, steps). We must add a "channels" dimension of 1.
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

print(f"Data loaded. X_train shape (after reshape): {X_train.shape}")

Loading data...
Data loaded. X_train shape (after reshape): (8000, 100, 1)


In [10]:
print("Defining Baseline (DeepSignal-like) model...")

# --- Module 1: CNN for raw signals ---
signal_input = Input(shape=(SIGNAL_WINDOW_SIZE, 1), name='signal_input')
cnn_features = Conv1D(filters=64, kernel_size=5, activation='relu')(signal_input)
cnn_features = Flatten()(cnn_features)

# --- Module 2: Bi-LSTM for sequence context ---
# The paper's architecture is complex. This is a simplified version
# that captures the *spirit* (CNN + RNN).
bilstm_features = Bidirectional(LSTM(32))(signal_input)

# --- Concatenate and Classify ---
combined_features = Concatenate()([cnn_features, bilstm_features])

dense_layer = Dense(64, activation='relu')(combined_features)
output = Dense(1, activation='sigmoid')(dense_layer) # Binary classification (0 or 1)

model_baseline = Model(inputs=signal_input, outputs=output)

# Compile the model
model_baseline.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

model_baseline.summary()

Defining Baseline (DeepSignal-like) model...


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ signal_input        │ (None, 100, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 96, 64)    │        384 │ signal_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 6144)      │          0 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_2     │ (None, 64)        │      8,704 │ signal_input[0][… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 6208)      │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │    397,376 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         65 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 406,529 (1.55 MB)

 Trainable params: 406,529 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
print("Training baseline model...")

# We train on a small number of epochs.
# The assignment says a "toy example is acceptable".
# This proves the *methodology*[cite: 39].
history_baseline = model_baseline.fit(
    X_train, 
    y_train, 
    validation_split=0.2, # Use 20% of training data for validation
    epochs=50, 
    batch_size=32
)

print("\nBaseline model training complete.")

# Save the trained model
model_baseline.save('baseline_model.keras')
print("Baseline model saved to 'baseline_model.keras'")

Training baseline model...
Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.5964 - auc: 0.6306 - loss: 0.6669 - val_accuracy: 0.6625 - val_auc: 0.7283 - val_loss: 0.6226
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6980 - auc: 0.7636 - loss: 0.5839 - val_accuracy: 0.7275 - val_auc: 0.8146 - val_loss: 0.5345
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.7767 - auc: 0.8545 - loss: 0.4858 - val_accuracy: 0.7906 - val_auc: 0.8817 - val_loss: 0.4495
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8533 - auc: 0.9267 - loss: 0.3662 - val_accuracy: 0.8575 - val_auc: 0.9290 - val_loss: 0.3524
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9084 - auc: 0.9658 - loss: 0.2613 - val_accuracy: 0.8881 - val_auc: 0.9547 - val_loss: 0.2859
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9370 - auc: 0.9839 - loss: 0.1880 - val_accuracy: 0.9000 - val_auc: 0.9685 - val_loss: 0.2411
Epoch 7

In [12]:
print("Evaluating baseline model on the test set...")
results_baseline = model_baseline.evaluate(X_test, y_test)

print(f"Test Loss: {results_baseline[0]}")
print(f"Test Accuracy: {results_baseline[1]}")
print(f"Test AUC: {results_baseline[2]}")

Evaluating baseline model on the test set...
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9055 - auc: 0.9509 - loss: 0.5221
Test Loss: 0.5221409797668457
Test Accuracy: 0.9054999947547913
Test AUC: 0.9509173631668091
